In [7]:
import numpy as np
from tensorflow import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation
from kerastuner.tuners import RandomSearch
from tensorflow.keras import Input, Model
import tensorflow as tf



def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = normalize(X_train)
X_test = normalize(X_test)
X_train = X_train.reshape(-1,32, 32, 3)  # reshaping for convnet


y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

def build_model(hp):  # random search passes this hyperparameter() object 

    inp = Input(shape=(32, 32, 3), name='inp')
    f1 = hp.Float('f1', min_value=0.0, max_value=0.2, step=0.1)
    x = tf.keras.layers.experimental.preprocessing.RandomRotation(f1)(inp)
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)

    x = Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3), input_shape=X_train.shape[1:], 
               activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'))(inp)

    x = MaxPooling2D(pool_size=(2, 2)) (x)

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        x = Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3),
                   activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'))(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    for i in range(hp.Int('n_connections', 1, 4)):
        x = Dense(hp.Choice(f'n_nodes',
                                  values=[128, 256, 512, 1024]),
                        activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'))(x)

    out = Dense(10, activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'))(x)


    model = Model(inputs=[inp], outputs=[out])
    
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'adagrad', 'SGD']),
                  loss="categorical_crossentropy",
                  metrics=["acc"])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=3,  # how many model variations to test?
    executions_per_trial=2,  # how many trials per variation? (same model could perform differently)
    directory='CNN_DeepLearning',
    project_name='cnn_best_1')

# Number of epochs
NUM_EPOCH = 5

tuner.search(x=X_train,
             y=y_train,
             epochs=NUM_EPOCH,
             batch_size=32,
             validation_split=0.2)

print(tuner.get_best_models()[0].summary())
print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

model = tuner.get_best_models(num_models=1)[0]
print (model.summary())
# Evaluate the best model.
loss, accuracy = model.evaluate(X_test, y_test)
print('loss:', loss)
print('accuracy:', accuracy)
model.save('cnn_best.h5')

Trial 3 Complete [01h 28m 47s]
val_acc: 0.1863499954342842

Best val_acc So Far: 0.3385000079870224
Total elapsed time: 03h 34m 27s
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inp (InputLayer)            [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 96)        2688      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 128)       110720    
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 32)        36896     
                                                             

None
{'f1': 0.1, 'input_units': 96, 'dense_activation': 'relu', 'n_layers': 2, 'conv_0_units': 128, 'n_connections': 2, 'n_nodes': 256, 'optimizer': 'adam', 'conv_1_units': 32}
Results summary
Results in CNN_DeepLearning/cnn_best_1
Showing 10 best trials
Trial summary
Hyperparameters:
f1: 0.1
input_units: 96
dense_activation: relu
n_layers: 2
conv_0_units: 128
n_connections: 2
n_nodes: 256
optimizer: adam
conv_1_units: 32
Score: 0.3385000079870224
Trial summary
Hyperparameters:
f1: 0.0
input_units: 224
dense_activation: relu
n_layers: 3
conv_0_units: 160
n_connections: 2
n_nodes: 512
optimizer: SGD
conv_1_units: 64
conv_2_units: 32
Score: 0.1863499954342842
Trial summary
Hyperparameters:
f1: 0.1
input_units: 96
dense_activation: tanh
n_layers: 2
conv_0_units: 256
n_connections: 1
n_nodes: 512
optimizer: adagrad
conv_1_units: 128
Score: 0.11454999819397926
None
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape     

In [6]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
